In [1]:
# Download data fro3m http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
# %lprun -f

In [2]:
import numpy as np
import re
import collections
import operator
from scipy.optimize import minimize
import copy
import cvxopt
import scipy.io as sio

from scipy.optimize import fmin_bfgs

In [3]:
def findDic(text):
    words = re.compile('[a-zA-Z\']*[a-zA-Z]').findall(text)
    return collections.Counter(words)

In [4]:
def readData(directory, paths, limit=None):
    if limit:
        paths = paths[0:limit]

    data = []
    for path in paths:
        txt = open(directory + path).read()
        data.append(txt)
    return data
        
        
negDataPath = !ls data/neg/
negData = readData('./data/neg/', negDataPath, 1000)

posDataPath = !ls data/pos/
posData = readData('./data/pos/', posDataPath, 1000)

In [5]:
# negData = [
#     'I am exhausted of this work.',
#     "I can't cooperate with this",
#     'I do not enjoy my job',
#     "I feel brilliant!",
#     'Gary is a friend of mine.',
#     'The date was good.'
# ]
# posData = [
#     'I love this burger.',
#     'This is an brilliant place!',
#     'I feel very good about these dates.',
#     'This is my best work.',
#     "What an awesome view"
# ]

# test = [
#     "I am not feeling well today.",
#     "I can't believe I'm doing this.",
#     'The date was good.',
#     "I am not feeling well today.",
#     "I can't believe I'm doing this.",
#     'I do not enjoy my job'
# ]

In [6]:
data = sio.loadmat('data/mpqa/bigram_mpqa.mat')

In [7]:
data_samples = []
for line in open('data/mpqa/mpqa_t4.dat', 'r'):
    data_samples.append(line.split())

In [8]:
x = [' '.join(line[1:]) for line in data_samples]
y = [int(line[0]) for line in data_samples]

In [9]:
posData = [x[i] for i in xrange(len(x)) if y[i] == 1]
negData = [x[i] for i in xrange(len(x)) if y[i] == 0]

In [10]:
print negData[0]
print "+++++"
print posData[0]

complaining
+++++
are also being encouraged


In [11]:
negData = negData[2000:]
posData = posData[2000:]

In [12]:
testNegData = negData[:2000]
testPosData = posData[:2000]

In [13]:
def findWordCount(data):
    wordCount = []
    for review in data:
        wordCount.append(findDic(review))
    return wordCount

negDataDic = findWordCount(negData)
posDataDic = findWordCount(posData)

In [14]:
def updateDic(mainDic, dicArray):
    mainDic = copy.deepcopy(mainDic)
    for dic in dicArray:
        mainDic.update(dic)
    
    # update indexes
    for index, key in enumerate(mainDic):
        mainDic[ key ] = index
        
    return mainDic

allWords = {}
allWords = updateDic(allWords, negDataDic)
allWords = updateDic(allWords, posDataDic)

In [15]:
def findRow(words, text, boolean=False):
    row = np.zeros(len(allWords))
    wordsDic = findDic(text)
    for word in words:
        if word in allWords:
            row[allWords[word]] = 1 if boolean else wordsDic[word]
    return row

In [16]:
def createTable(allWords, negData, y0, posData, y1):
    x = []
    for data in negData + posData:
        x.append( findRow(allWords, data) )
    x = np.row_stack(x)
    
    y = np.append(y0 * np.ones(len(negData)), y1 * np.ones(len(posData)))
    return x, y

In [17]:
x, y = createTable(allWords, negData, -1, posData, 1)

In [18]:
M = len(x)
k = np.zeros( (M, M) )

In [19]:
M

6604

In [20]:
for i in xrange(M):
    for j in xrange(M):
        k[i][j] = np.dot(x[i].T, x[j])

In [158]:
P = cvxopt.matrix(np.outer(y, y) * k)
q = cvxopt.matrix(np.ones(M) * -1)

G = cvxopt.matrix(np.vstack([
    np.eye(M) * -1,
    np.eye(M)
    ]))

h = cvxopt.matrix(np.hstack([
    np.zeros(M),
    np.ones(M) * 999999999.0
    ]))

A = cvxopt.matrix(y, (1,M))
b = cvxopt.matrix(0.0)

solution = cvxopt.solvers.qp(P, q, G, h, A, b)

     pcost       dcost       gap    pres   dres
 0:  1.5661e+20 -4.2776e+21  1e+22  7e-01  1e-05
 1:  1.9048e+20 -2.0508e+21  3e+21  2e-01  1e+01
 2:  1.1907e+20 -4.6981e+20  7e+20  2e-02  2e+00
 3:  5.0741e+19 -1.8691e+20  3e+20  6e-03  5e-01
 4:  1.8904e+19 -7.5548e+19  1e+20  8e-04  5e-02
 5:  7.7127e+18 -3.2811e+19  4e+19  3e-04  1e-02
 6:  2.3405e+18 -9.4332e+18  1e+19  1e-03  2e-03
 7:  1.0259e+18 -4.6222e+18  6e+18  6e-04  2e-04
 8:  3.1434e+17 -1.3750e+18  2e+18  1e-03  4e-05
 9:  1.2304e+17 -5.4971e+17  7e+17  9e-04  5e-06
10:  6.5836e+16 -2.9724e+17  4e+17  5e-04  2e-06
11:  2.6967e+16 -1.1326e+17  1e+17  3e-04  1e-06
12:  1.2498e+16 -5.5409e+16  7e+16  1e-04  1e-06
13:  3.0831e+15 -1.0573e+16  1e+16  3e-04  1e-06
14:  5.8228e+14 -1.3307e+15  2e+15  2e-04  1e-06
15:  9.5118e+13 -1.4032e+14  2e+14  4e-04  1e-06
16:  1.3980e+13 -1.8418e+13  3e+13  7e-05  1e-06
17:  2.0601e+12 -3.6425e+12  6e+12  3e-04  1e-06
18:  5.6187e+11 -1.5642e+12  2e+12  8e-05  2e-06
19:  1.7546e+11 -9.15

In [159]:
a = solution['x']

In [160]:
findRow(allWords, test[0])

NameError: name 'test' is not defined

In [ ]:
a

In [ ]:
w = np.zeros(len(x[0]))

In [ ]:
for i in xrange(len(a)):
    w += a[i] * y[i] * x[i]

In [ ]:
w

In [ ]:
w.dot( findRow(allWords, negData[1]) )

In [ ]:
negMax = None
for i in negData:
    t = w.dot( findRow(allWords, negData[1]) )
    if not negMax or t > negMax:
        negMax = t

In [ ]:
posMin = None
for i in posData:
    t = w.dot( findRow(allWords, posData[1]) )
    if not posMin or t < posMin:
        posMin = t

In [ ]:
b = (negMax+posMin)/2

In [ ]:
b

In [ ]:
w.dot( findRow(allWords, negData[0]) )

In [ ]:
correct = 0
error_pos = []
for i in testPosData:
    if w.dot( findRow(allWords, i) ) > 0:
        correct += 1
for i in testNegData:
    if w.dot( findRow(allWords, i) ) < 0:
        correct += 1

In [ ]:
correct*1. / (len(testPosData) + len(testNegData))

In [ ]:
(len(posData) + len(negData))

In [162]:
import re
re.compile('^[01]').findall('065 fdsafad 0fasf asf0asdf0sad0')

['0']